In [42]:
# sudo -H anaconda/bin/pip install biopython

from Bio import Entrez

In [43]:
Entrez.email = 'greg.clinton@gmail.com'

In [ ]:
"""
How many genes does e. coli have? What are their names?
How many species under homo?
sra vs nucleotide, gene, ref_seq
"""

In [44]:
record = Entrez.read(Entrez.esearch(db = 'gene', term = 'e coli[Orgn]'))

In [49]:
ids = ','.join(record['IdList'][:5])
ids

'947170,948466,947777,948511,947071'

In [50]:
records = Entrez.read(Entrez.efetch(db = 'gene', id = ids, retmode = 'xml'))
len(records)

5

In [52]:
print records[0].keys()

[u'Entrezgene_rna', u'Entrezgene_unique-keys', u'Entrezgene_track-info', u'Entrezgene_comments', u'Entrezgene_locus', u'Entrezgene_source', u'Entrezgene_type', u'Entrezgene_gene-source', u'Entrezgene_xtra-iq', u'Entrezgene_non-unique-keys', u'Entrezgene_gene', u'Entrezgene_properties']


In [58]:
print records[0]['Entrezgene_type']

2
